## Feature selection with vanilla linear models

The multi-step feature selection approach of AutoFeat is based on multiple calls to an L1-regularized linear model to select the features in several rounds. For this, the linear model needs to be trained quickly and select reliable features in terms of precision (only relevant features, not too much junk) and recall (all relevant features). In this notebook, several sklearn models are benchmarked to see which model should be used for AutoFeat.

#### Regression
`ElasticNet` and `Lasso` are very similar, so are `Lars` and `LassoLars`. `LassoLarsIC` sometimes does not select any features. `OrthogonalMatchingPursuit` is very fast but also often selects too few features. With a good trade-off in terms of speed and selected features, we use the `LassoLarsCV` model for AutoFeat.

#### Classification
We tested `linear_model.LogisticRegressionCV` and `svm.LinearSVC` (together with a grid search), however, Logistic Regression is much faster, therefore it does not make sense to use the SVC.

In [ ]:
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
from time import time
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from nlputils.visualize import get_colors

from autofeat import AutoFeatModel

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
def create_plots(precision, recall, noise_levels, noise_feat_frac, n_train, n_feat_true):
    colors = get_colors(len(noise_levels))
    plt.figure()
    for i, noise in enumerate(noise_levels):
        plt.plot(noise_feat_frac, precision[i, :], c=colors[i], label="noise: %g" % noise)
    plt.legend(bbox_to_anchor=(1., 1.))
    plt.xlabel("# noise features / # training samples ($n$)")
    plt.title("Precision ($n$: %i; $d$: %i)" % (n_train, n_feat_true))
    plt.figure()
    for i, noise in enumerate(noise_levels):
        plt.plot(noise_feat_frac, recall[i, :], c=colors[i], label="noise: %g" % noise)
    plt.legend(bbox_to_anchor=(1., 1.))
    plt.xlabel("# noise features / # training samples ($n$)")
    plt.title("Recall ($n$: %i; $d$: %i)" % (n_train, n_feat_true))
    colors = get_colors(len(noise_feat_frac))
    plt.figure()
    for i, nfeat in enumerate(noise_feat_frac):
        plt.plot(noise_levels, precision[:, i], c=colors[i], label="noise feat frac: %g" % nfeat)
    plt.legend(bbox_to_anchor=(1., 1.))
    plt.xlabel("noise level")
    plt.title("Precision ($n$: %i; $d$: %i)" % (n_train, n_feat_true))
    plt.figure()
    for i, nfeat in enumerate(noise_feat_frac):
        plt.plot(noise_levels, recall[:, i], c=colors[i], label="noise feat frac: %g" % nfeat)
    plt.legend(bbox_to_anchor=(1., 1.))
    plt.xlabel("noise level")
    plt.title("Recall ($n$: %i; $d$: %i)" % (n_train, n_feat_true))
    plt.figure()
    plt.imshow(precision)
    plt.xticks(list(range(len(noise_feat_frac))), noise_feat_frac)
    plt.xlabel("# noise features / # training samples ($n$)")
    plt.yticks(list(range(len(noise_levels))), noise_levels)
    plt.ylabel("noise level")
    plt.clim(0, 1)
    plt.colorbar()
    plt.title("Precision ($n$: %i; $d$: %i)" % (n_train, n_feat_true))
    plt.figure()
    plt.imshow(recall)
    plt.xticks(list(range(len(noise_feat_frac))), noise_feat_frac)
    plt.xlabel("# noise features / # training samples ($n$)")
    plt.yticks(list(range(len(noise_levels))), noise_levels)
    plt.ylabel("noise level")
    plt.clim(0, 1)
    plt.colorbar()
    plt.title("Recall ($n$: %i; $d$: %i)" % (n_train, n_feat_true))

def prec_rec(coefs, true_features):
    # sort weights by absolute values
    w_dict = dict(zip(range(len(coefs)), np.abs(coefs)))
    sorted_features = sorted(w_dict, key=w_dict.get, reverse=True)
    # --> how many of the true features are amongst the len(true_features) highest weights?
    n_selected = int(min(len(true_features), np.sum(np.abs(coefs)>0)))
    recall = len(set(sorted_features[:n_selected]).intersection(true_features)) / len(true_features)
    # --> how many features do we need to take to get all true features?
    min_thr = min(w_dict[t] for t in true_features)
    precision = len(true_features) / np.sum(np.abs(coefs)>= min_thr)
    return precision, recall

def prec_rec_autofeat(selected_features, true_features):
    if not len(selected_features):
        return 0, 0
    TP = len(set(selected_features).intersection(true_features))
    recall =  TP / len(true_features)
    precision = TP / len(selected_features)
    return precision, recall

def get_dataset(n_train=100, n_feat_noise=100, n_feat_true=10, noise=0.01, ptype="regression", random_seed=None):
    if random_seed is not None:
        np.random.seed(random_seed)
    # create a simple problem with 1 target variable,
    # generated as a linear combination of random features
    # and include some additional noise filters
    X = np.random.randn(n_train, n_feat_noise+n_feat_true)
    true_features = np.random.permutation(n_feat_noise+n_feat_true)[:n_feat_true]
    y = X[:, true_features].sum(axis=1)
    if ptype == "regression":
        # add normally distributed noise
        y -= y.mean()
        y /= y.std()
        y = (1-noise)*y + noise*np.random.randn(len(y))
        #y = y/y.std() + noise*np.random.randn(len(y))
    else:
        # threshold to transform into classification problem
        y = np.array(y > y.mean(), dtype=int)
        # randomly flip some idx
        flip_idx = np.random.permutation(len(y))[:int(np.ceil(len(y)*noise))]
        y[flip_idx] -= 1
        y = np.abs(y)
    return X, y, true_features

def compute_autofeat_dataset(n_train=100):
    np.random.seed(10)
    x1 = np.random.rand(n_train)
    x2 = np.random.randn(n_train)
    x3 = np.random.rand(n_train)
    y = 2 + 15*x1 + 3/(x2 - 1/x3) + 5*(x2 + np.log(x1))**3
    X = pd.DataFrame(np.vstack([x1, x2, x3]).T, columns=["x1", "x2", "x3"])
    # generate new features with autofeat
    afreg = AutoFeatModel(verbose=1, feateng_steps=3, featsel_runs=-1, problem_type=None)
    X = afreg.fit_transform(X, y)
    return X, y

def get_autofeat_dataset(n_feat_noise=100, noise=0.01, ptype="regression", random_seed=None):
    if random_seed is not None:
        np.random.seed(random_seed)
    true_features = ["x1", "(x2 + log(x1))**3", "1/(x2 - 1/x3)"]
    noise_features = list(np.random.permutation(list(set(df.columns).difference(true_features)))[:n_feat_noise])
    X = StandardScaler().fit_transform(df[true_features + noise_features])
    y = target - target.mean()
    y /= y.std()
    if ptype == "regression":
        # add normally distributed noise
        y = (1-noise)*y + noise*np.random.randn(len(y))
    else:
        # threshold to transform into classification problem
        y = np.array(y > y.mean(), dtype=int)
        # randomly flip some idx
        flip_idx = np.random.permutation(len(y))[:int(np.ceil(len(y)*noise))]
        y[flip_idx] -= 1
        y = np.abs(y)
    return X, y, [0, 1, 2]

In [ ]:
# compute the original autofeat dataset with all features
df, target = compute_autofeat_dataset(n_train=100)

In [ ]:
# independent features
ptype = "regression"
t0 = time()
n_train = 100
n_feat_true = 10
noise_levels = [0., 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.75, 1.]
noise_feat_frac = [0.01, 0.05, 0.1, 0.25, 0.5, 1, 2, 5, 10, 25]
precision = np.zeros((len(noise_levels), len(noise_feat_frac)))
recall = np.zeros((len(noise_levels), len(noise_feat_frac)))
for i, noise in enumerate(noise_levels):
    print("noise level: %g" % noise)
    for j, nfeat in enumerate(noise_feat_frac):
        ps, rs = [], []
        for seed in range(10):
            X, y, true_features = get_dataset(n_train, int(nfeat*n_train), n_feat_true, noise, ptype, random_seed=seed)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                if ptype == "regression":
                    # model = lm.OrthogonalMatchingPursuitCV(cv=5, max_iter=min(X.shape[1], 20)).fit(X, y)
                    model = lm.LassoLarsCV(cv=5).fit(X, y)
                    coefs = model.coef_
                else:
                    model = lm.LogisticRegressionCV(cv=5, penalty="l1", solver="saga", class_weight="balanced").fit(X, y)  
                    coefs = np.max(np.abs(model.coef_), axis=0)
            p, r = prec_rec(coefs, true_features)
            ps.append(p)
            rs.append(r)
        precision[i, j] = np.mean(ps)
        recall[i, j] = np.mean(rs)
print("took %.1f sec" % (time() - t0))

create_plots(precision, recall, noise_levels, noise_feat_frac, n_train, n_feat_true)

In [ ]:
# autofeat features
ptype = "regression"
t0 = time()
n_train = 100
n_feat_true = 3
noise_levels = [0., 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.75, 1.]
noise_feat_frac = [0.01, 0.05, 0.1, 0.25, 0.5, 1, 2, 5, 10, 25]
precision = np.zeros((len(noise_levels), len(noise_feat_frac)))
recall = np.zeros((len(noise_levels), len(noise_feat_frac)))
for i, noise in enumerate(noise_levels):
    print("noise level: %g" % noise)
    for j, nfeat in enumerate(noise_feat_frac):
        ps, rs = [], []
        for seed in range(10):
            X, y, true_features = get_autofeat_dataset(int(nfeat*n_train), noise, ptype, random_seed=seed)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                if ptype == "regression":
                    # model = lm.OrthogonalMatchingPursuitCV(cv=5, max_iter=min(X.shape[1], 20)).fit(X, y)
                    model = lm.LassoLarsCV(cv=5).fit(X, y)
                    coefs = model.coef_
                else:
                    model = lm.LogisticRegressionCV(cv=5, penalty="l1", solver="saga", class_weight="balanced").fit(X, y)
                    coefs = np.max(np.abs(model.coef_), axis=0)
                selected_features = np.where(coefs>1e-8)[0]
            p, r = prec_rec_autofeat(selected_features, true_features)
            ps.append(p)
            rs.append(r)
        precision[i, j] = np.mean(ps)
        recall[i, j] = np.mean(rs)
print("took %.1f sec" % (time() - t0))

create_plots(precision, recall, noise_levels, noise_feat_frac, n_train, n_feat_true)

In [ ]:
# multi-class classification
X, y = make_classification(2000, 5000, 20, 100, 0, 5, random_state=10)

In [ ]:
%%time
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = lm.LogisticRegressionCV(cv=5, penalty="l1", solver="saga", class_weight="balanced").fit(X, y)
    print(model.score(X, y))

In [ ]:
%%time 
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    param_grid = {"C": np.logspace(-4, 4, 10)}
    clf = svm.LinearSVC(penalty="l1", class_weight="balanced", dual=False)
    model = GridSearchCV(clf, param_grid, cv=5)
    model.fit(X, y)
    print(model.score(X, y))